In [1]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.initializers import Zeros
import sklearn.datasets as datasets
import pickle

In [2]:
CENTERS = [[0, 0]]
DATA, _ = datasets.make_blobs(
        n_samples=200,
        centers=CENTERS,
        cluster_std=1,
        random_state=1
    )

cw1 = -1 
cw2 = 1
cb = -1

sw1 = -1
sw2 = 1
sb = 1

X = np.array(list(filter(lambda x : cw1 * x[0] + cw2 * x[1] + cb < -.5 or cw1 * x[0] + cw2 * x[1] + cb > .5, DATA)))
X = np.array(list(filter(lambda x : sw1 * x[0] + sw2 * x[1] + sb < -.5 or sw1 * x[0] + sw2 * x[1] + sb > .5, X)))
C = np.array([0 if cw1 * x[0] + cw2 * x[1] + cb >= 0 else 1 for x in X])
S = np.array([0 if sw1 * x[0] + sw2 * x[1] + sb >= 0 else 1 for x in X])
pass_fail = C ^ S


In [3]:
# Build the model using Input layer
inputs = keras.layers.Input(shape=(2,))
hidden = keras.layers.Dense(3, activation="sigmoid")(inputs)  # Hidden layer with 3D output
outputs = keras.layers.Dense(1, activation="sigmoid")(hidden)  # Output layer
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-1), loss="binary_crossentropy")
# Prepare storage for hidden space data
hidden_space_trajectory = []
original_points = X  # Store the original points

hidden_layer_model = keras.Model(inputs=model.input, outputs=model.layers[1].output)

2025-03-16 16:47:28.534547: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-16 16:47:28.534762: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-03-16 16:47:28.534778: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1742158048.535310   75324 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742158048.535633   75324 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Train the model for all epochs and record the hidden space every 10 epochs
epochs_per_interval = 10
total_epochs = 100
hidden_space_trajectory = []

for epoch in range(1, total_epochs + 1):
    # Train for one epoch at a time to monitor progress
    model.fit(X, pass_fail, epochs=1, verbose=1)

    # Record the hidden space trajectory every 10 epochs
    if epoch % epochs_per_interval == 0:
        # Inspect weights
        weights = model.layers[1].get_weights()[0]  # Weights of the hidden layer
        print(f"Weights after {epoch} epochs:\n{weights}")
        hidden_positions = hidden_layer_model.predict(X)
        hidden_space_trajectory.append(hidden_positions)
        print(f"Recorded hidden space at epoch {epoch}")

# Convert hidden_space_trajectory to numpy array for storage
hidden_space_trajectory = np.array(hidden_space_trajectory)
print(f"Hidden space trajectory shape: {hidden_space_trajectory.shape}")

2025-03-16 16:47:33.949575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - loss: 0.7054
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6894 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6911
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6836 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6700 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6419 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6154 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5652
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5007 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4036
Weights after 10 epochs:
[[ 1.2438341  2.1792872  2.7604434]
 [-2.1147068 -2.133245  -2.6547651]]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Recorded hidden space at epoch 10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3616
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3197
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2682 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2268
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1844 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/s

In [5]:
# Save the trajectory and original points to a pickle file
with open("hidden_space_and_points.pkl", "wb") as f:
    pickle.dump({"hidden_space_trajectory": hidden_space_trajectory, "original_points": original_points}, f)

print(f"Hidden space trajectory saved with shape: {hidden_space_trajectory.shape}")
print(f"Original points saved with shape: {original_points.shape}")

Hidden space trajectory saved with shape: (10, 115, 3)
Original points saved with shape: (115, 2)
